In [ ]:
!pip install pyngrok
from flask import Flask, request, jsonify
from pyngrok import ngrok
import ast
import json

# **Initialize the LLM**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
model_name = "google/flan-t5-xxl"

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto" 
)

tokenizer = AutoTokenizer.from_pretrained(model_name)


# **Flask server**

In [ ]:
app = Flask(__name__)
@app.route('/def', methods=['Get'])
def c():
    return 'Hello'
@app.route('/contacts', methods=['POST'])
def contacts():
    data = request.get_json()
    contacts_text = data.get('text','No text found')
    print(data)
    contacts_prompt = f"""
    You are an assistant inside the **Contacts** screen of a mobile app.
    
    Your task is to understand what the user wants and return their request as this exact format:
    
    action: <action>, name: <name or null>, phone: <phone or null>
    
    ### Valid actions:
    - add
    - view
    - call
    - dial
    
    ### Examples:
    
    User: "Add Sarah, her number is 6912345678."
    Output: action: add, name: Sarah, phone: 6912345678
    
    User: "Call my brother Tom."
    Output: action: call, name: Tom, phone: null
    
    User: "Open the dialer please."
    Output: action: dial, name: null, phone: null
    
    User: "Show me all my contacts."
    Output: action: view, name: null, phone: null
    
    User: "Make a call now."
    Output: action: call, name: null, phone: null
    
    User: "Call John."
    Output: action: call, name: John, phone: null
    
    User: "Dial the number pad."
    Output: action: dial, name: null, phone: null
    
    ### Now respond to this user request:
    "{contacts_text}"
    Output: 
    """
    
    inputs = tokenizer(contacts_prompt, return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_length=150,
        num_beams=5,
        temperature=0.0,
        early_stopping=True,
        no_repeat_ngram_size=2
    )
    
    contacts_result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(contacts_result)
    contacts_list = [part.strip() for part in contacts_result.split(",")]
    print(contacts_list)
    contacts_result_dict = dict(item.split(": ", 1) for item in contacts_list)
    for key in ["action", "name", "phone"]:
        contacts_result_dict.setdefault(key, "null")
    if "phone" in contacts_result_dict:
        contacts_result_dict["phone"] = re.sub(r"[ -]", "", contacts_result_dict["phone"])
    if contacts_result_dict["name"] not in contacts_text:
        contacts_result_dict["name"] = "null"
    json_str = json.dumps(contacts_result_dict)
    print(json_str)
    
    return json_str
@app.route('/add-edit-contact', methods=['POST'])
def addcontact():
    data = request.get_json()
    addcontact_text = data.get('text','No text found')
    print(data)
    addcontact_prompt = f"""
    You are an assistant inside the **Add Contact** screen of a mobile app.
    
    Your task is to understand the name and the phone of the contact that the user wants to add and if it is an emergency contact:
    
    name: <name or null>, phone: <phone or null>, ergency: <true/false or null>
    
    If you cant find a name or phone or emergency you can set them to null.
    
    ### Examples:
    
    User: "Add Sarah, her number is 6912345678."
    Output: name: Sarah, phone: 6912345678, emergency: null
    
    User: "6912342232."
    Output: name: null, phone: 6912342232, emergency: null
    
    User: "The name is Mark"
    Output: name: Mark, phone: null, emergency: null

    User: "The name is Mark and it is an emergency contact"
    Output: name: Mark, phone: null, emergency: true
    
    ### Now respond to this user request:
    "{addcontact_text}"
    Output: 
    """
    inputs = tokenizer(addcontact_prompt, return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_length=150,
        num_beams=5,
        temperature=0.0,
        early_stopping=True,
        no_repeat_ngram_size=2
    )
    
    addcontact_result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(addcontact_result)
    addcontact_list = [part.strip() for part in addcontact_result.split(",")]
    print(addcontact_list)
    addcontact_result_dict = dict(item.split(": ", 1) for item in addcontact_list)
    for key in ["name", "phone","emergency"]:
        addcontact_result_dict.setdefault(key, "null")
    if "phone" in addcontact_result_dict:
        addcontact_result_dict["phone"] = re.sub(r"[ -]", "", addcontact_result_dict["phone"])
    if "emergency" not in addcontact_text.lower():
        addcontact_result_dict["emergency"] = "null"
    json_str = json.dumps(addcontact_result_dict)
    print(json_str)
    return json_str

@app.route('/viewcontact', methods=['POST'])
def viewcontact():
    viewdata = request.get_json()
    viewcontact_text = viewdata.get('text','No text found')
    print(viewdata)
    viewcontact_prompt = f"""
    You are inside the **View Contact** screen of a mobile app.
    
    The user speaks a command, and your task is to detect the intended action.
    
    Valid actions:
    - call
    - delete
    - edit
    
    Respond with exactly one of these actions, depending on what the user clearly said.
    If the user didn’t clearly ask to call, delete, or edit the contact, respond with:
    action: null
    
    ### Format:
    action: <call/delete/edit/null>
    
    ### Examples:
    
    User: "Can you call this person?"
    Output: action: call
    
    User: "I want to remove this contact"
    Output: action: delete
    
    User: "Make changes to the contact"
    Output: action: edit
    
    User: "I need to update the number"
    Output: action: edit
    
    User: "Just checking the contact"
    Output: action: "null"
    
    User: "Who is this?"
    Output: action: "null"
    
    User: "Delete it now!"
    Output: action: delete
    
    ### Now respond to this user request:
    "{viewcontact_text}"
    Output:
    """

    inputs = tokenizer(viewcontact_prompt, return_tensors="pt").to("cuda")
        
    outputs = model.generate(
        **inputs,
        max_length=150,
        num_beams=5,
        temperature=0.0,
        early_stopping=True,
        no_repeat_ngram_size=2
    )
    
    view_result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(view_result)
    viewcontact_list = [part.strip() for part in view_result.split(",")]
    print(viewcontact_list)
    viewcontact_result_dict = dict(item.split(": ", 1) for item in viewcontact_list)
    json_str = json.dumps(viewcontact_result_dict)
    print(json_str)
    return json_str

@app.route('/soscall', methods=['POST'])
def soscall():
    sosdata = request.get_json()
    sos_text = sosdata.get('text','No text found')
    print(sosdata)
    sos_prompt = f"""
    You are inside the **SOS Emergency Call** screen of a mobile app.
    
    The user gives a voice command. Your task is to identify **which emergency service** they want to contact.
    
    Valid services:
    - ambulance
    - police
    - firedepartment
    
    If the user clearly asks for one of these services, respond with just the service name.
    If it's unclear or unrelated, respond with:
    null
    
    ### Examples:
    
    User: "Call an ambulance now!"
    Output: ambulance
    
    User: "I need the police"
    Output: police
    
    User: "There’s a fire in the building"
    Output: firedepartment
    
    User: "Emergency, call the fire department!"
    Output: firedepartment
    
    User: "Help me"
    Output: null
    
    User: "What's this app?"
    Output: null
    
    ### Now respond to this user request:
    "{sos_text}"
    Output:
    """

    inputs = tokenizer(sos_prompt, return_tensors="pt").to("cuda")
        
    outputs = model.generate(
        **inputs,
        max_length=150,
        num_beams=5,
        temperature=0.0,
        early_stopping=True,
        no_repeat_ngram_size=2
    )
    
    sos_result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(sos_result)
    sos_result_dict = {"service":sos_result}
    json_str = json.dumps(sos_result_dict)
    print(json_str)
    return json_str

